In [1]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd
import pymeshlab as pml
import time
import matplotlib as plt
import sys
sys.path.append('../preprocessing')
from viz import viz_mesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:

import pandas as pd
df = pd.read_pickle("../querying/normalized_features_final.pkl")
k = 10
npdata = df[df.columns[2:9]].to_numpy()
for col in df.columns[9:]:
    npcol = df[col].to_numpy()
    npdata = np.hstack((npdata,np.array([a for a in npcol])))
labels = df["class"].to_numpy()


features_total = npdata
paths = (df["path"].str.replace("./features","../resampledPML") + ".obj").to_numpy()

In [16]:
npdata.shape

(2475, 507)

In [53]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler


# Assuming 'labels' is your string label array
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Now, you can one-hot encode the integer labels
labels_one_hot = to_categorical(labels_encoded)


# scaler = StandardScaler()
# X_normalized = scaler.fit_transform(npdata)  # Step 1: Standardization
# scalar_X_normalized = scaler.fit_transform(scalar_features)

# Split the dataset into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(npdata, labels_one_hot, test_size=0.2,  random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5,  random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(scalar_features, labels_one_hot, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
indices = np.arange(len(npdata))

# Use train_test_split with 'indices' to split your data while maintaining index correspondence
indices_train, indices_test = train_test_split(indices, test_size=0.2, random_state=42)
indices_test, indices_val = train_test_split(indices_test, test_size=0.5, random_state=42)

import tensorflow as tf

X_train.shape[1]
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation="softmax")
])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) i used by mistake binary_crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

np.argmax(X_train)
history = model.fit(npdata, labels_one_hot, epochs=60, batch_size=48, validation_data=(X_val, y_val))


Epoch 1/60
52/52 [==============================] - 4s 27ms/step - loss: 3.8402 - accuracy: 0.1091 - val_loss: 3.4124 - val_accuracy: 0.1371
Epoch 2/60
52/52 [==============================] - 0s 8ms/step - loss: 3.1850 - accuracy: 0.2004 - val_loss: 2.9398 - val_accuracy: 0.2016
Epoch 3/60
52/52 [==============================] - 1s 12ms/step - loss: 2.8418 - accuracy: 0.2440 - val_loss: 2.7255 - val_accuracy: 0.2702
Epoch 4/60
52/52 [==============================] - 1s 14ms/step - loss: 2.6389 - accuracy: 0.2869 - val_loss: 2.5518 - val_accuracy: 0.2984
Epoch 5/60
52/52 [==============================] - 0s 9ms/step - loss: 2.4425 - accuracy: 0.3196 - val_loss: 2.3883 - val_accuracy: 0.3226
Epoch 6/60
52/52 [==============================] - 1s 13ms/step - loss: 2.3159 - accuracy: 0.3499 - val_loss: 2.3227 - val_accuracy: 0.3347
Epoch 7/60
52/52 [==============================] - 0s 9ms/step - loss: 2.2089 - accuracy: 0.3838 - val_loss: 2.2002 - val_accuracy: 0.3911
Epoch 8/60
52/52

In [50]:
# shufflenpdata = np.append(np.append(X_test,X_val),X_train).reshape(-1,507)
# shufflelabel = np.append(np.append(y_test,y_val),y_train).reshape(-1,69)

In [55]:
model.evaluate(X_test,y_test)

8/8 [==============================] - 0s 8ms/step - loss: 0.4434 - accuracy: 0.8664


[0.4434220790863037, 0.8663967847824097]

In [54]:
totalpreds = model.predict(npdata)

78/78 [==============================] - 1s 4ms/step


In [66]:
query_object = totalpreds[10]
# Calculate distances using Euclidean distance as an example
distances = np.linalg.norm(totalpreds - query_object, axis=1)

# Combine distances and labels into a list of tuples
objects_with_distances = list(zip(distances, totalpreds, labels))

# Sort the objects based on distances
objects_with_distances.sort(key=lambda x: x[0])

# Get the 5 closest objects and their labels
closest_objects = objects_with_distances[:10]

# Extract the labels from the closest objects
closest_labels = [obj[2] for obj in closest_objects]

print("Closest Labels:", closest_labels)

Closest Labels: ['AircraftBuoyant', 'Rocket', 'Rocket', 'Rocket', 'Rocket', 'Rocket', 'Rocket', 'Rocket', 'Rocket', 'Rocket']


In [71]:
query_objects = totalpreds[100:200]
top_k = 10
accuracies = []

# Loop over all query objects
for query_object in query_objects:
    # Calculate distances using Euclidean distance as an example
    distances = np.linalg.norm(totalpreds - query_object, axis=1)

    # Combine distances and labels into a list of tuples
    objects_with_distances = list(zip(distances, totalpreds, labels))

    # Sort the objects based on distances
    objects_with_distances.sort(key=lambda x: x[0])

    # Get the 10 closest objects
    closest_objects = objects_with_distances[1:top_k+1]
    true_label = objects_with_distances[0][2]
    # Extract the labels from the closest objects
    closest_labels = [obj[2] for obj in closest_objects]

    # Count how many of the closest objects have the same label as the query object
    same_label_count = closest_labels.count(true_label)

    # Calculate the accuracy for this query object
    accuracy = same_label_count / top_k  # Since we are considering the top 10 closest objects
    accuracies.append(accuracy)

# Calculate the overall accuracy
overall_accuracy = np.mean(accuracies)

print("Accuracy for Each Query Object:", accuracies)
print("Overall Accuracy:", overall_accuracy)

Accuracy for Each Query Object: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4, 0.9, 1.0, 1.0, 0.8, 0.8, 0.1, 1.0, 0.3, 1.0, 0.9, 0.0, 1.0, 0.9, 1.0, 0.9, 1.0, 1.0, 0.9, 0.8, 0.9, 0.9, 0.5, 0.8, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 0.8, 0.9, 0.8, 1.0, 0.0, 0.9, 1.0, 0.9, 1.0, 1.0, 1.0, 0.2, 0.9, 0.9, 0.9, 0.8, 0.9, 0.9, 0.0, 0.2, 0.9, 0.4, 0.9, 0.9, 0.8, 0.9, 0.8, 0.9, 0.9, 0.9, 0.9, 0.5, 0.6, 0.9, 0.7, 0.0, 0.8, 0.8, 0.9, 1.0, 0.9, 0.8, 0.8, 0.8, 0.9, 1.0]
Overall Accuracy: 0.8420000000000001
